In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import matplotlib.pyplot as plt


In [ ]:
# !unzip -q /kaggle/input/dogs-vs-cats/train.zip
# !unzip -q /kaggle/input/dogs-vs-cats/test1.zip

In [ ]:
directory ='/kaggle/working/finalhairdataset'
os.makedirs(directory)

In [ ]:
# import os

# directory = 'C:\Users\anmol\Downloads\archive\data0330'

# prefix = 'bald'
# i=100
# for filename in os.listdir(directory):
#     if filename.endswith('.jpg'):
#         old_path = os.path.join(directory, filename)
#         new_filename = prefix+'.'+ str(i)
#         new_path = os.path.join('C:\Users\anmol\OneDrive\Desktop', new_filename)
# #         os.rename(old_path, new_path)
#         i=i+1


In [ ]:
from sklearn.model_selection import train_test_split
filenames = os.listdir("/kaggle/input/semifinalharidataset/Dataset/Train")

categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'bald':
        categories.append('bald')
    elif category == 'avg':
#         print(filename)
        categories.append('avg')
    elif category =='avgt':
        print(filename)
        categories.append('avg')
    else:
        categories.append('notbald')

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})
train_df, val_df = train_test_split(df, test_size=0.20, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = val_df.reset_index(drop=True)

In [ ]:
print(df.head
     )

In [ ]:
image_size = (128, 128)
train_image_generator = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
train_generator = train_image_generator.flow_from_dataframe(
    train_df, 
    "/kaggle/input/semifinalharidataset/Dataset/Train", 
    x_col='filename',
    y_col='category',
    target_size=image_size,
    class_mode='categorical',
    batch_size=16
)
val_image_generator = ImageDataGenerator(rescale=1./255)
val_generator = val_image_generator.flow_from_dataframe(
    val_df, 
    "/kaggle/input/semifinalharidataset/Dataset/Train", 
    x_col='filename',
    y_col='category',
    target_size=image_size,
    class_mode='categorical',
    batch_size=16
)

In [ ]:
def build_model():
    width, height, channels = 128, 128, 3
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, channels)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))


    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    
    model.add(Dense(3, activation='softmax')) 

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model
cnn = build_model()

In [ ]:
history = cnn.fit(
        train_generator, 
        epochs=10,
        validation_data=val_generator,
        validation_steps=5000//16,
        steps_per_epoch=20000//16,)


In [ ]:
cnn.save('cnn3.h5')
cnn.save_weights('cnnweights3.h5')


In [ ]:

plt.plot(history.history['accuracy'], color='b', label="Training accuracy")
plt.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
plt.xlim(0, 9)
plt.ylim(0, 1)
legend = plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
results={
    0:'bald',
    1:'avg',
    2:'notbald'
}
from PIL import Image
import numpy as np
im=Image.open("")
im=im.resize((128,128))
im=np.expand_dims(im,axis=0)
im=np.array(im)
im=im/255
pred = cnn.predict(im)
predicted_class = np.argmax(pred[0])
print(predicted_class, results[predicted_class])
